In [3]:
import dspy
from dspy.datasets.gsm8k import GSM8K, gsm8k_metric

# Import the optimizer
from dspy.teleprompt import MIPROv2

# Initialize the LM
lm = dspy.LM('openai/gpt-4o-mini', api_key='YOUR_OPENAI_API_KEY')
dspy.configure(lm=lm)

# Initialize optimizer
teleprompter = MIPROv2(
    metric=gsm8k_metric,
    auto="medium", # Can choose between light, medium, and heavy optimization runs
)

# Optimize program
print(f"Optimizing program with MIPROv2...")
gsm8k = GSM8K()
optimized_program = teleprompter.compile(
    dspy.ChainOfThought("question -> answer"),
    trainset=gsm8k.train,
)

# Save optimize program for future use
optimized_program.save(f"optimized.json")

Optimizing program with MIPROv2...


README.md: 0.00B [00:00, ?B/s]

main/train-00000-of-00001.parquet:   0%|          | 0.00/2.31M [00:00<?, ?B/s]

main/test-00000-of-00001.parquet:   0%|          | 0.00/419k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/7473 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1319 [00:00<?, ? examples/s]

100%|██████████| 1319/1319 [00:00<00:00, 40012.78it/s]
2025/09/02 16:57:10 INFO dspy.teleprompt.mipro_optimizer_v2: 
RUNNING WITH THE FOLLOWING MEDIUM AUTO RUN SETTINGS:
num_trials: 18
minibatch: True
num_fewshot_candidates: 12
num_instruct_candidates: 6
valset size: 160

2025/09/02 16:57:10 INFO dspy.teleprompt.mipro_optimizer_v2: 
==> STEP 1: BOOTSTRAP FEWSHOT EXAMPLES <==
2025/09/02 16:57:10 INFO dspy.teleprompt.mipro_optimizer_v2: These will be used as few-shot example candidates for our program and for creating instructions.

2025/09/02 16:57:10 INFO dspy.teleprompt.mipro_optimizer_v2: Bootstrapping N=12 sets of demonstrations...


Bootstrapping set 1/12
Bootstrapping set 2/12
Bootstrapping set 3/12


  0%|          | 0/40 [00:00<?, ?it/s]2025/09/02 16:57:19 WARNING dspy.adapters.json_adapter: Failed to use structured output format, falling back to JSON mode.
2025/09/02 16:57:23 ERROR dspy.teleprompt.bootstrap: Failed to run or to evaluate example Example({'question': "The result from the 40-item Statistics exam Marion and Ella took already came out. Ella got 4 incorrect answers while Marion got 6 more than half the score of Ella. What is Marion's score?", 'gold_reasoning': "Ella's score is 40 items - 4 items = <<40-4=36>>36 items. Half of Ella's score is 36 items / 2 = <<36/2=18>>18 items. So, Marion's score is 18 items + 6 items = <<18+6=24>>24 items.", 'answer': '24'}) (input_keys={'question'}) with <function gsm8k_metric at 0x7365a171d300> due to litellm.AuthenticationError: AuthenticationError: OpenAIException - Incorrect API key provided: YOUR_OPE*******_KEY. You can find your API key at https://platform.openai.com/account/api-keys..
  2%|▎         | 1/40 [00:13<08:43, 13.41s/

KeyboardInterrupt: 

In [2]:
from pydantic import BaseModel
from typing import Literal
import dspy
import mipro

# Define the Dish class
class Dish(BaseModel):
    # Output
    dish_type: Literal["Fish", "Vegetarian", "Chicken"] = dspy.OutputField()
    name: str = dspy.OutputField()

# Define the Meal signature
class Meal(dspy.Signature):
    """
    Suggest a meal
    """
    # Outputs
    dish: Dish = dspy.OutputField()
    day_of_the_week: Literal[
        "Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"
    ] = dspy.OutputField()

# Initialize the base language model
lm = dspy.OpenAI(model="gpt-3.5-turbo")

# Create a dataset of example inputs/outputs
examples = [
    Meal(dish=Dish(dish_type="Fish", name="Grilled Salmon"), day_of_the_week="Monday"),
    Meal(dish=Dish(dish_type="Vegetarian", name="Vegetable Stir Fry"), day_of_the_week="Tuesday"),
    Meal(dish=Dish(dish_type="Chicken", name="Chicken Alfredo"), day_of_the_week="Wednesday"),
]

# Initialize the MIPROv2 optimizer
optimizer = mipro.MIPROv2(signature=Meal, lm=lm, trainset=examples)

# Optimize the prompt
optimized_prompt = optimizer.optimize()

# Print the optimized prompt or configuration
print("Optimized Prompt Configuration:")
print(optimized_prompt)


ModuleNotFoundError: No module named 'mipro'

In [1]:
import dspy
from dspy.evaluate import MIPROv2
from pydantic import BaseModel
from typing import Literal

# Define the structured output model
class Dish(BaseModel):
    dish_type: Literal["Fish", "Vegetarian", "Chicken"] = dspy.OutputField()
    name: str = dspy.OutputField()

# Define the DSPy signature without any inputs
class Meal(dspy.Signature):
    """
    Suggest a meal
    """
    dish: Dish = dspy.OutputField()
    day_of_the_week: Literal["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"] = dspy.OutputField()

# Define a validation metric
def validate_match(expected: Meal, actual: Meal) -> bool:
    return (
        expected.dish.dish_type == actual.dish.dish_type and
        expected.dish.name == actual.dish.name and
        expected.day_of_the_week == actual.day_of_the_week
    )

# Create a small trainset
trainset = [
    Meal(dish={"dish_type": "Fish", "name": "Grilled Salmon"}, day_of_the_week="Monday"),
    Meal(dish={"dish_type": "Vegetarian", "name": "Lentil Soup"}, day_of_the_week="Tuesday"),
    Meal(dish={"dish_type": "Chicken", "name": "Chicken Alfredo"}, day_of_the_week="Wednesday"),
    Meal(dish={"dish_type": "Vegetarian", "name": "Chickpea Curry"}, day_of_the_week="Thursday"),
    Meal(dish={"dish_type": "Fish", "name": "Fish Tacos"}, day_of_the_week="Friday"),
]

# Define the model to be optimized
class MealGenerator(dspy.Module):
    def __init__(self):
        self.prompt = dspy.Predict(Meal)

    def forward(self):
        return self.prompt()

# Run MIPROv2 optimization
mipro = MIPROv2(metric=validate_match, trainset=trainset, num_candidates=5)
optimized_model = mipro.optimize(MealGenerator())

# Test the optimized model
result = optimized_model.forward()
print(result)

ImportError: cannot import name 'MIPROv2' from 'dspy.evaluate' (/usr/local/python/3.12.1/lib/python3.12/site-packages/dspy/evaluate/__init__.py)

In [ ]:
import dspy
from dspy.evaluate import MIPROv2
from pydantic import BaseModel
from typing import Literal

# 🔧 Configure two language models
generation_lm = dspy.OpenAI(model="gpt-4")         # Used for generating meals
judge_lm = dspy.OpenAI(model="gpt-3.5-turbo")       # Used for evaluating predictions

# Set the default LM for generation
dspy.configure(model=generation_lm)

# 🧠 Define domain model
class Dish(BaseModel):
    dish_type: Literal["Fish", "Vegetarian", "Chicken"] = dspy.OutputField()
    name: str = dspy.OutputField()

class Meal(dspy.Signature):
    """
    Suggest a meal
    """
    dish: Dish = dspy.OutputField()
    day_of_the_week: Literal[
        "Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"
    ] = dspy.OutputField()

# ✅ Metric using judge LM
def validate_match(expected: Meal, actual: Meal) -> bool:
    with dspy.context(judge_lm):
        return (
            expected.dish.dish_type == actual.dish.dish_type and
            expected.dish.name == actual.dish.name and
            expected.day_of_the_week == actual.day_of_the_week
        )

# 📊 Trainset
trainset = [
    Meal(dish={"dish_type": "Fish", "name": "Grilled Salmon"}, day_of_the_week="Monday"),
    Meal(dish={"dish_type": "Vegetarian", "name": "Lentil Soup"}, day_of_the_week="Tuesday"),
    Meal(dish={"dish_type": "Chicken", "name": "Chicken Alfredo"}, day_of_the_week="Wednesday"),
    Meal(dish={"dish_type": "Vegetarian", "name": "Chickpea Curry"}, day_of_the_week="Thursday"),
    Meal(dish={"dish_type": "Fish", "name": "Fish Tacos"}, day_of_the_week="Friday"),
]

# 🔁 Model to optimize
class MealGenerator(dspy.Module):
    def __init__(self):
        self.prompt = dspy.Predict(Meal)

    def forward(self):
        return self.prompt()

# 🚀 Run MIPROv2 optimization
mipro = MIPROv2(metric=validate_match, trainset=trainset, num_candidates=5)
optimized_model = mipro.optimize(MealGenerator())

# 🧪 Test the optimized model
result = optimized_model.forward()
print(result)